## Problem

ある工場で、鉄鋼、電力、労働力という３種類のリソースを使って、２種類の製品を生産しており、いま、来週の生産計画を立てようとしています。 

製品単位当たりの生産に必要な各リソースの量、リソース許容上限は下表に示した通りです。

利益が最大となるように、各製品の生産量を決めてください。

|        | 製品1 | 製品2 | 上限 |
|     -  |     - |     - |    - |
|   鉄鋼 |     1 |     2 |   14 |
|   電力 |     1 |     1 |    8 |
| 労働力 |     3 |     1 |   18 |
|   利益 |     2 |     3 |      |

(http://www.morito.mgmt.waseda.ac.jp/koukyou2005/how2solver.pdf)


In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
    columns=["製品1", "製品2", "上限"],
    index=["鉄鋼", "電力", "労働力", "利益"],
    data=[
        [1, 2, 14],
         [1, 1, 8],
         [3, 1, 18],
         [2, 3]
    ]
)
df

,製品1,製品2,上限
鉄鋼,1,2,14.0
電力,1,1,8.0
労働力,3,1,18.0
利益,2,3,NaN


In [3]:
problem = pulp.LpProblem(name="production_planning", sense=pulp.LpMaximize)
problem

production_planning:
MAXIMIZE
None
VARIABLES

In [4]:
products = df.columns[:-1].values.tolist()
resource = df.index[:-1].values.tolist()
print(products, resource)
table = pulp.LpVariable.dicts("", (products), cat=pulp.LpInteger, lowBound=0)
table

['製品1', '製品2'] ['鉄鋼', '電力', '労働力']


{'製品1': _製品1, '製品2': _製品2}

In [5]:
problem += pulp.lpSum([df[p]["利益"] * table[p] for p in products])
problem

production_planning:
MAXIMIZE
2*_製品1 + 3*_製品2 + 0
VARIABLES
0 <= _製品1 Integer
0 <= _製品2 Integer

In [6]:
for r in resource:
    problem += pulp.lpSum(df[p][r] * table[p] for p in table.keys()) <= df["上限"][r]
problem

production_planning:
MAXIMIZE
2*_製品1 + 3*_製品2 + 0
SUBJECT TO
_C1: _製品1 + 2 _製品2 <= 14

_C2: _製品1 + _製品2 <= 8

_C3: 3 _製品1 + _製品2 <= 18

VARIABLES
0 <= _製品1 Integer
0 <= _製品2 Integer

In [7]:
status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status", pulp.LpStatus[status])

Status Optimal


In [8]:
print("=" * 3, "Result", "=" * 29)
print('利益: {}'.format(pulp.value(problem.objective)))
print("-" * 40)
print("生産量内訳")
for p in products:
    print(table[p].value())
print("=" * 40)

=== Result =============================
利益: 22.0
----------------------------------------
生産量内訳
2.0
6.0
